In [5]:
import json
import re
import uuid
from tqdm import tqdm

In [6]:
loss_pattern = r'.*?\n-{4,}'

In [7]:
def clean_loss(new_context_list):
    idx = 0
    final_context_list = []
    while idx < len(new_context_list)-1:
        curr_res = re.findall(loss_pattern, new_context_list[idx])
        if curr_res:
            next_res = re.findall(loss_pattern, new_context_list[idx+1])
            if next_res:
                #print("mv.......", new_context_list[idx])
                idx += 1
            else:   
                #print("adding1:",new_context_list[idx])
                final_context_list.append(new_context_list[idx])
                idx += 1
        else:
            final_context_list.append(new_context_list[idx])
            #print("adding2:",new_context_list[idx])
            idx += 1
        #print(final_context_list)
    final_context_list.append(new_context_list[-1])  
    return final_context_list

def clean_yangsheng(context):
    #result = re.findall(noise_pattern, context)
    #context = re.sub(noise_pattern, "\n\n",context)
    #context = context.replace("\n\n*   **", "\n\n**")
    split_token = "\n\n"
    new_context_list = []
    context_list = context.split(split_token)
    for item in context_list:
        if "文章" not in item:
            new_context_list.append(item)
        else:
            break
    if new_context_list[-1] == "别名：":
        new_context_list = new_context_list[:-1]
    if len(new_context_list[-1].replace("=","").strip("\n")) == 0:
        new_context_list = new_context_list[:-1]

    new_context_list = clean_loss(new_context_list)
    last_line = new_context_list[-1]
    res = re.findall(loss_pattern, last_line)
    if res:
        new_context_list = new_context_list[:-1]
    new_context = split_token.join(new_context_list)
    return new_context.strip(" ").strip("\n").strip(" ").strip("\n")

In [9]:
fw = open("../../../../full_data/zhonghuayangsheng/clean.jsonl","w",encoding="utf-8")
all_data = {}
with open("../../../../full_data/zhonghuayangsheng/all_data_preformat.jsonl", "r",encoding="utf-8") as fs:
    for item in tqdm(fs.readlines()):
        item = json.loads(item)
        context = item["text"]
        context = clean_yangsheng(context)
        item["text"] = context
        item = json.dumps(item, ensure_ascii=False)
        fw.write(item + "\n")

100%|██████████| 694/694 [00:00<00:00, 12715.29it/s]
